In [2]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

sns.set_style()
plt.rcParams['figure.figsize'] = (12, 6)

In [3]:
vars_procedimientos = {f"PROCEDIMIENTO{i}": str for i in range(1, 31)}

In [4]:
df_procesada = pd.read_csv("../data/processed/df_procesada.csv", sep=";", dtype=vars_procedimientos)

C:\Users\ppizarro\AppData\Local\Temp\ipykernel_7140\2020992259.py:1: DtypeWarning: Columns (24,25,26,27,28,29,30,31,32,33,37,41,43,47,51,75,76,77,78,79,80,81,82,83,84,85,86,87,123) have mixed types. Specify dtype option on import or set low_memory=False.
  df_procesada = pd.read_csv("../data/processed/df_procesada.csv", sep=";", dtype=vars_procedimientos)


In [5]:
COLUMNAS_SERVICIO_TRASLADO = [
    "SERVICIOINGRESO": 0,
    "SERVICIOTRASLADO1": 1,
    "SERVICIOTRASLADO2": 2,
    "SERVICIOTRASLADO3": 3,
    "SERVICIOTRASLADO4": 4,
    "SERVICIOTRASLADO5": 5,
    "SERVICIOTRASLADO6": 6,
    "SERVICIOTRASLADO7": 7,
    "SERVICIOTRASLADO8": 8,
    "SERVICIOTRASLADO9": 9,
    "SERVICIOALTA": 10,
]

COLUMNAS_FECHA_TRASLADO = {
    "FECHA_INGRESO": 0,
    "FECHATRASLADO1": 1,
    "FECHATRASLADO2": 2,
    "FECHATRASLADO3": 3,
    "FECHATRASLADO4": 4,
    "FECHATRASLADO5": 5,
    "FECHATRASLADO6": 6,
    "FECHATRASLADO7": 7,
    "FECHATRASLADO8": 8,
    "FECHATRASLADO9": 9,
    "FECHAALTA": 10,
}


# Filtra la base nacional para dejar el Torax
df_torax = df_procesada.query("COD_HOSPITAL == 112103")
# Permite identificar cada egreso
df_torax = df_torax.reset_index().reset_index(names="id_egreso")

In [10]:
df_torax[COLUMNAS_FECHA_TRASLADO]

,FECHA_INGRESO,FECHATRASLADO1,FECHATRASLADO2,FECHATRASLADO3,FECHATRASLADO4,FECHATRASLADO5,FECHATRASLADO6,FECHATRASLADO7,FECHATRASLADO8,FECHATRASLADO9,FECHAALTA
0,2019-01-14,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019-01-17
1,2019-08-26,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019-08-26
2,2019-01-09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019-01-09
3,2019-02-18,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019-02-18
4,2019-06-18,2019-06-29,2019-06-30,2019-07-02,NaN,NaN,NaN,NaN,NaN,NaN,2019-07-05
...,...,...,...,...,...,...,...,...,...,...,...
13461,2021-07-15,2021-07-24,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021-07-26
13462,2021-07-03,2021-07-08,2021-07-18,2021-07-20,2021-07-30,NaN,NaN,NaN,NaN,NaN,2021-07-30
13463,2021-07-26,2021-07-30,2021-08-05,2021-08-23,NaN,NaN,NaN,NaN,NaN,NaN,2021-09-05
13464,2021-10-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021-11-04


In [14]:
df_torax.melt(id_vars="id_egreso", value_vars=COLUMNAS_FECHA_TRASLADO).sort_values(["id_egreso", "variable"])

,id_egreso,variable,value
134660,0,FECHAALTA,2019-01-17
13466,0,FECHATRASLADO1,NaN
26932,0,FECHATRASLADO2,NaN
40398,0,FECHATRASLADO3,NaN
53864,0,FECHATRASLADO4,NaN
...,...,...,...
94261,13465,FECHATRASLADO6,NaN
107727,13465,FECHATRASLADO7,NaN
121193,13465,FECHATRASLADO8,NaN
134659,13465,FECHATRASLADO9,NaN
